# Lesson 1 Demo 0: PostgreSQL and AutoCommits

<img src="images/postgresSQLlogo.png" width="250" height="250">

## Walk through the basics of PostgreSQL autocommits 

In [2]:
!pip install psycopg2

     ---------------------------------------- 1.2/1.2 MB 5.0 MB/s eta 0:00:00


In [1]:
# IMPORT POSTGRES PYTHON ADAPTER
import psycopg2

ModuleNotFoundError: No module named 'psycopg2'

### Create a connection to the database
1. Connect to the local instance of PostgreSQL (*127.0.0.1*)
2. Use the database/schema from the instance. 
3. The connection reaches out to the database (*studentdb*) and use the correct privilages to connect to the database (*user and password = student*).

In [ ]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

### Use the connection to get a cursor that will be used to execute queries.

In [ ]:
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

### Create a database to work in

In [ ]:
try: 
    cur.execute("select * from test")
except psycopg2.Error as e:
    print(e)

### Error occurs, but it was to be expected because table has not been created as yet. To fix the error, create the table. 

In [ ]:
try:
    cur.execute("CREATE TABLE test (col1 int, col2 int, col3 int);")
except psycopg2.Error as e:
    print(e)

### Error indicates we cannot execute this query. Since we have not committed the transaction and had an error in the transaction block, we are blocked until we restart the connection. Hence, we close our connection to the default database, reconnect to the Udacity database, and get a new cursor

In [ ]:
try: 
    conn.close()
except psycopg2.Error as e:
    print(e)

try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
    
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

In our exercises instead of worrying about commiting each transaction or getting a strange error when we hit something unexpected, let's set autocommit to true. **This says after each call during the session commit that one action and do not hold open the transaction for any other actions. One action = one transaction.**

In this demo we will use automatic commit so each action is commited without having to call `conn.commit()` after each command. **The ability to rollback and commit transactions are a feature of Relational Databases.**

In [ ]:
conn.set_session(autocommit=True)

In [ ]:
cur.execute("select * from test")

In [ ]:
cur.execute("CREATE TABLE test (col1 int, col2 int, col3 int);")

### Once autocommit is set to true, we execute this code successfully. There were no issues with transaction blocks and we did not need to restart our connection. 

In [ ]:
try: 
    cur.execute("select count(*) from test")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
print(cur.fetchall())

### INSERT INTO TABLE

In [ ]:
try: 
    cur.execute("INSERT INTO test (col1 col2, col3) \
                 VALUES (%s, %s, %s)", \
                 (1, 23, 1970))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("INSERT INTO test (col1 col2, col3) \
                 VALUES (%s, %s, %s)", \
                 (2, 39, 1969))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

#### Validate your data was inserted into the table.
The while loop is used for printing the results. If executing queries in the Postgres shell, this would not be required.

Note: If you run the insert statement code more than once, you will see duplicates of your data. PostgreSQL allows for duplicate

In [ ]:
try: 
    cur.execute("SELECT * FROM music_library;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

#### Drop the table to avoid duplicates and clean up

In [ ]:
try: 
    cur.execute("DROP table music_library")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)

#### Close the curson and connection

In [ ]:
cur.close()
conn.close()